In [ ]:
import numpy as np
import pandas as pd
import vectorbtpro as vbt

In [ ]:
## Load m1 data
m1_data = vbt.HDFData.fetch('../../data/GU_OHLCV_3Y.h5')
m1_data.wrapper.index #pandas doaesn't recognise the frequency because of missing timestamps

In [ ]:
m15_data = m1_data.resample('15T')  # Convert 1 minute to 15 mins
h1_data = m1_data.resample("1h")    # Convert 1 minute to 1 hour
h4_data = m1_data.resample('4h')    # Convert 1 minute to 4 hour

In [ ]:
m15_data.wrapper.index

In [ ]:
# Obtain all the closing  prices using the .get() method
m15_close = m15_data.get()['Close']


## h1 data
h1_open  = h1_data.get()['Open']
h1_close = h1_data.get()['Close']
h1_high  = h1_data.get()['High']
h1_low   = h1_data.get()['Low']

## h4 data
h4_open  = h4_data.get()['Open']
h4_close = h4_data.get()['Close']
h4_high  = h4_data.get()['High']
h4_low   = h4_data.get()['Low']

### Create ( _manually_ ) the indicators for Multi-TimeFrames

In [ ]:
rsi_period = 21

## 15m indicators
m15_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(m15_data.get("Close"), skipna=True).real
m15_bbands = vbt.talib("BBANDS").run(m15_data.get("Close"), skipna=True)
m15_bbands_rsi = vbt.talib("BBANDS").run(m15_rsi, skipna=True)


## h4 indicators
h4_rsi = vbt.talib("RSI", timeperiod = rsi_period).run(h4_data.get("Close"), skipna=True).real
h4_bbands = vbt.talib("BBANDS").run(h4_data.get("Close"), skipna=True)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi, skipna=True)

### Creating `vbt.Resampler()` for `Upsampling`
Upsampling means resampling a higher timeframe (`low frequency`) time series data to lower timeframe (`high frequency`) time series data

In [ ]:
def create_resamplers(result_dict_keys_list : list, source_indices : list,  
                      source_frequencies :list, target_index : pd.Series, target_freq : str):
    """
    Creates a dictionary of vbtpro resampler objects.

    Parameters
    ==========
    result_dict_keys_list : list, list of strings, which are keys of the output dictionary
    source_indices        : list, list of pd.time series objects of the higher timeframes
    source_frequencies    : list, llist(str), which are short form representation of time series order. Eg:["1D", "4h"]
    target_index          : pd.Series, target time series for the resampler objects
    target_freq           : str, target time frequency for the resampler objects

    Returns
    ===========
    resamplers_dict       : dict, vbt pro resampler objects
    """
    
    
    resamplers = []
    for si, sf in zip(source_indices, source_frequencies):
        resamplers.append(vbt.Resampler(source_index = si,  target_index = target_index,
                                        source_freq = sf, target_freq = target_freq))
    return dict(zip(result_dict_keys_list, resamplers))

In [ ]:
## Create Resampler Objects for upsampling
src_indices = [m15_close.index, h4_close.index]
src_frequencies = ["15T", "4h"] 
resampler_dict_keys = ["m15_m15", "h4_m15"]

list_resamplers = create_resamplers(resampler_dict_keys, src_indices, src_frequencies, m15_close.index, "15T")

list_resamplers

In [ ]:
## Initialize  dictionary
data = {}

## Use along with  Manual indicator creation method for MTF
series_to_resample = [
    [m15_close, m15_rsi, m15_bbands.upperband, m15_bbands.middleband, m15_bbands.lowerband, 
    m15_bbands_rsi.upperband, m15_bbands_rsi.middleband, m15_bbands_rsi.lowerband],
    [h4_close, h4_rsi, h4_bbands.upperband, h4_bbands.middleband, h4_bbands.lowerband, 
    h4_bbands_rsi.upperband, h4_bbands_rsi.middleband, h4_bbands_rsi.lowerband],
    ]

data_keys = [
    ["m15_close", "m15_rsi", "m15_bband_price_upper",  "m15_bband_price_middle", "m15_bband_price_lower",  
                              "m15_bband_rsi_upper",  "m15_bband_rsi_middle", "m15_bband_rsi_lower"], 
    ["h4_close", "h4_rsi", "h4_bband_price_upper",  "h4_bband_price_middle",  "h4_bband_price_lower", 
                            "h4_bband_rsi_upper",  "h4_bband_rsi_middle", "h4_bband_rsi_lower" ],
         ]

In [ ]:
## Use this along with Method 1 of indicator creation

for lst_series, lst_keys, resampler in zip(series_to_resample, data_keys, resampler_dict_keys):
    for key, time_series in zip(lst_keys, lst_series):
        resampled_time_series = time_series.vbt.resample_closing(list_resamplers[resampler])
        data[key] = resampled_time_series.ffill()

In [ ]:
## Add h4 OLH data - No need to do ffill() on resample_closing as it already does that by default

data["h4_open"] = h4_open.vbt.resample_opening(list_resamplers['h4_m15'])#.ffill()
data["h4_high"] = h4_high.vbt.resample_closing(list_resamplers['h4_m15'])#.ffill()
data["h4_low"]  = h4_low.vbt.resample_closing(list_resamplers['h4_m15'])#.ffill()
data["h4_close"] = h4_open.vbt.resample_closing(list_resamplers['h4_m15'])#.ffill()

In [ ]:
cols_order = ['m15_close', 'm15_rsi', 'm15_bband_price_upper','m15_bband_price_middle', 'm15_bband_price_lower',
              'm15_bband_rsi_upper','m15_bband_rsi_middle', 'm15_bband_rsi_lower',
              'h4_open', 'h4_high', 'h4_low', 'h4_close', 'h4_rsi',
              'h4_bband_price_upper', 'h4_bband_price_middle', 'h4_bband_price_lower', 
              'h4_bband_rsi_upper', 'h4_bband_rsi_middle', 'h4_bband_rsi_lower'
 ]

In [ ]:
## construct a multi-timeframe dataframe
mtf_df = pd.DataFrame(data)[cols_order]
print("Length of mtf_df:",len(mtf_df), f'on {mtf_df.index.freq} frequency')
display(mtf_df)

In [ ]:
for col in mtf_df.columns:
    time_series = mtf_df[col]
    print(col,time_series.index.freq, 'length:' ,len(time_series), 'NULL Count:',time_series.isna().sum())

In [ ]:
mtf_df.info()

### Double Bollinger Band - Strategy Conditions
The trading conditions (rules) of the strategy are as follows:

1. A long (buy) signal is generated whenever the H4 market (Low) price goes below its lower Bollinger band, and the 15m RSI goes below its lower Bollinger band.

2. A short (sell) signal is generated whenever the H4 market (High) price breaks its upper Bollinger band, and the 15m RSI breaks above its upper Bollinger band.

In [ ]:
required_cols = ['m15_close','m15_rsi','m15_bband_rsi_lower', 'm15_bband_rsi_upper',
                 'h4_low', "h4_rsi", "h4_bband_price_lower", "h4_bband_price_upper" ]

In [ ]:
## Higher values greater than 1.0 are like moving up the lower RSI b-band, 
## signifying if the lowerband rsi is anywhere around 1% of the lower b-band validate that case as True
bb_upper_fract = 0.99
bb_lower_fract = 1.01

## Long Entry Conditions
c1_long_entry = (mtf_df['h4_low'] <= mtf_df['h4_bband_price_lower'])
c2_long_entry = (mtf_df['m15_rsi'] <= (bb_lower_fract * mtf_df['m15_bband_rsi_lower']) )


## Long Exit Conditions
c1_long_exit =  (mtf_df['h4_high'] >= mtf_df['h4_bband_price_upper'])
c2_long_exit = (mtf_df['m15_rsi'] >= (bb_upper_fract * mtf_df['m15_bband_rsi_upper'])) 

In [ ]:
mtf_df[required_cols][c1_long_entry]

In [ ]:
mtf_df[required_cols][c2_long_entry]

In [ ]:
## Strategy conditions check - Using m15 and h4 data 
mtf_df['entry'] = c1_long_entry & c2_long_entry
mtf_df['exit']  = c1_long_exit & c2_long_exit

In [ ]:
mtf_df['signal'] = 0   
mtf_df['signal'] = np.where( mtf_df['entry'], 1, 0)
mtf_df['signal'] = np.where( mtf_df['exit'] , -1, mtf_df['signal'])

### Run Portfolio backtesting Simulation using `pf.from_signals()`

In [ ]:
entries = mtf_df.signal == 1.0
exits = mtf_df.signal == -1.0

In [ ]:
pf = vbt.Portfolio.from_signals(
    close = mtf_df['m15_close'], 
    entries = entries, 
    exits = exits, 
    direction = "both", ## This setting trades both long and short signals
    freq = pd.Timedelta(minutes=5), 
    init_cash = 100000
)

In [ ]:
pf.stats()

In [ ]:
pf.trade_history

In [ ]:
## Global Plot Settings
vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280

### Plotting - Portfolio Simulations

In [ ]:
pf.plot().show() ## This takes slightly long (10 secs) as it uses 15m timeframe
# pf.resample("1d").plot().show()

In [ ]:
## We can also isolate pf.orders from the above pf.plot
pf.orders.resample("1d").plot(xaxis=dict(rangeslider_visible=False),**{"title_text" : "Orders - Stats & Plot", 
                                                                       "title_font_size" : 18}).show()

In [ ]:
print(f"Max Drawdown [%]: {pf.stats()['Max Drawdown [%]']}")
print(f"Max Drawdown Duration: {pf.stats()['Max Drawdown Duration']}")
## Drawdown plot below shows top 5 drawdowns and 94 days of max drawdown duration includes
## 73 days for the declination phasd and 21 days for the recovery phase the max. peak drawdown
pf.drawdowns.plot(**{"title_text" : "Drawdowns Plot"}).show()

In [ ]:
## Documentation Reference to adjust title : https://plotly.com/python/reference/layout/
pf.plot_underwater(**{"title_text" : "Underwater Plot",'title_x': 0.5}).show()

### Plotting - Indicators and visualizing strategy

In [ ]:
def plot_strategy(slice_lower : str, slice_upper: str, df : pd.DataFrame , rsi : pd.Series,
                         bb_price : vbt.indicators.factory, bb_rsi : vbt.indicators.factory,  
                         pf: vbt.portfolio.base.Portfolio,
                         show_legend : bool = True):
    """Creates a stacked indicator plot for the 2BB strategy.
    Parameters
    ===========
    slice_lower : str, start date of dataframe slice in yyyy.mm.dd format
    slice_upper : str, start date of dataframe slice in yyyy.mm.dd format
    df          : pd.DataFrame, containing the OHLCV data
    rsi         : pd.Series, rsi indicator time series in same freq as df
    bb_price    : vbt.indicators.factory.talib('BBANDS'), computed on df['close'] price
    bb_rsi      : vbt.indicators.factory.talib('BBANDS') computer on RSI
    pf          : vbt.portfolio.base.Portfolio, portfolio simulation object from VBT Pro
    show_legend : bool, switch to show or completely hide the legend box on the plot
    
    Returns
    =======
    fig         : plotly figure object
    """
    kwargs1 = {"title_text" : "H4 OHLCV with BBands on Price and RSI", 
               "title_font_size" : 18,
               "height" : 960,
               "legend" : dict(yanchor="top",y=0.99, xanchor="left",x= 0.1)}
    fig = vbt.make_subplots(rows=2,cols=1, shared_xaxes=True, vertical_spacing=0.1)
    ## Filter Data according to date slice
    df_slice = df[["Open", "High", "Low", "Close"]][slice_lower : slice_upper]
    bb_price = bb_price[slice_lower : slice_upper]
    rsi = rsi[slice_lower : slice_upper]
    bb_rsi = bb_rsi[slice_lower : slice_upper]

    ## Retrieve datetime index of rows where price data is NULL
    # retrieve the dates that are in the original datset
    dt_obs = df_slice.index.to_list()
    # Drop rows with missing values
    dt_obs_dropped = df_slice['Close'].dropna().index.to_list()
    # store  dates with missing values
    dt_breaks = [d for d in dt_obs if d not in dt_obs_dropped]

    ## Plot Figures
    df_slice.vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1),  fig=fig, **kwargs1) ## Without Range Slider
    rsi.rename("RSI").vbt.plot(add_trace_kwargs=dict(row=2, col=1), trace_kwargs = dict(connectgaps=True), fig=fig) 

    bb_line_style = dict(color="white",width=1, dash="dot")
    bb_price.plot(add_trace_kwargs=dict(row=1, col=1),fig=fig, **kwargs1,
                lowerband_trace_kwargs=dict(fill=None, name = 'BB_Price_Lower', connectgaps=True, line = bb_line_style), 
                upperband_trace_kwargs=dict(fill=None, name = 'BB_Price_Upper', connectgaps=True, line = bb_line_style),
                middleband_trace_kwargs=dict(fill=None, name = 'BB_Price_Middle', connectgaps=True) )

    bb_rsi.plot(add_trace_kwargs=dict(row=2, col=1),limits=(25, 75),fig=fig,
                lowerband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Lower', connectgaps=True,line = bb_line_style), 
                upperband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Upper', connectgaps=True,line = bb_line_style),
                middleband_trace_kwargs=dict(fill=None, name = 'BB_RSI_Middle', connectgaps=True, visible = False))
    
    ## Plots Long Entries / Exits and Short Entries / Exits
    # pf[slice_lower:slice_upper].plot_trade_signals(add_trace_kwargs=dict(row=1, col=1),fig=fig,
    #                                                plot_close=False, plot_positions="lines")

    ## Plot Trade Profit or Loss Boxes
    pf.trades.direction_long[slice_lower : slice_upper].plot(
                                        add_trace_kwargs=dict(row=1, col=1),fig=fig,
                                        plot_close = False,
                                        plot_markers = False
                                        )
                                        

    pf.trades.direction_short[slice_lower : slice_upper].plot(
                                            add_trace_kwargs=dict(row=1, col=1),fig=fig,
                                            plot_close = False,
                                            plot_markers = False
                                            )


    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
    fig.layout.showlegend = show_legend  
    # fig.write_html(f"2BB_Strategy_{slice_lower}_to_{slice_upper}.html")
    
    return fig

In [ ]:
slice_lower = '2019.11.01'
slice_higher = '2019.12.31'

fig = plot_strategy(slice_lower, slice_higher, h4_data.get(), h4_rsi, 
                           h4_bbands, h4_bbands_rsi, pf,
                           show_legend = True)

fig.show()

In [ ]:
## Inspecting PnL Series - series length filtered by notna() should equal to the nr. of trades
print(f"Total Nr. of Trades: {pf.stats()['Total Orders']}")
pnl_series = pf.trades.pnl.to_pd()
pnl_series[pnl_series.notna()]

In [ ]:
mtf_df["pnl"] = pnl_series
mtf_df